In [126]:
# import modules
import requests
import shutil
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import urllib
import pickle
import time
from __future__ import print_function

%matplotlib inline

In [2]:
api_key = '5eef6423b6f7cea2f5b6cae0838e38f9'

## Get Dictionary Mapping Genre_id to Genre_name

In [228]:
# API Key (Harry's)
payload = {
            'api_key': '5eef6423b6f7cea2f5b6cae0838e38f9'
         }

r = requests.get("https://api.themoviedb.org/3/genre/movie/list", params = payload).json()

In [229]:
genres = {}
for result in r['genres']:
    genres[result["id"]] = result["name"]

In [230]:
# get rid of u's in front of genre id's
for key in genres.keys():
    genres[key] = genres[key].encode('utf-8').replace(" ", "_")

In [231]:
genres

{12: 'Adventure',
 14: 'Fantasy',
 16: 'Animation',
 18: 'Drama',
 27: 'Horror',
 28: 'Action',
 35: 'Comedy',
 36: 'History',
 37: 'Western',
 53: 'Thriller',
 80: 'Crime',
 99: 'Documentary',
 878: 'Science_Fiction',
 9648: 'Mystery',
 10402: 'Music',
 10749: 'Romance',
 10751: 'Family',
 10752: 'War',
 10770: 'TV_Movie'}

In [131]:
# pickle genres
with open('genre_dict.pickle', 'wb') as handle:
    pickle.dump(genres, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Get Data from TMDb

In [ ]:
for genre in non_war_genres:
    # get data for genre
    genre_data = []
    for i in range(1, 7):
        url = "https://api.themoviedb.org/3/discover/movie"
        payload = {
                    'api_key': api_key,
                    'with_genres': str(genre),
                    'page': i
                 }
        r = requests.get(url, params = payload).json()
        genre_data += r["results"]
        time.sleep(1)
    # store the genre data
    name_genre = genres[genre]
    data[name_genre] = genre_data

## Pickle TMDb Data

In [ ]:
with open('data_by_genre.pickle', 'wb') as handle:
    pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

# test pickling
with open('data_by_genre.pickle', 'rb') as handle:
    data_pickle = pickle.load(handle)

## Create Directories to Store Posters

In [127]:
from os import makedirs
for genre_name in genres.values():
    makedirs("posters/" + genre_name)

## Get 100 Posters for each Genre based on TMDb Data

In [130]:
for genre, name_genre in genres.items():
# iterate through the genre data and store posters in appropriate directory
    print(name_genre)
    counter = 0
    for movie in data[name_genre]:
        if len(listdir("posters/" + name_genre)) == 100:
            break
        elif movie['poster_path']:
            url = 'http://image.tmdb.org/t/p/w500/' + movie['poster_path']
            
            # build filename
            title = movie["title"].encode('utf-8').replace("/", " ")
            genre_ids = "_".join(str(id) for id in movie["genre_ids"])
            movie_id = str(movie["id"])
            path = "posters"+ "/" + name_genre + "/" + title + "-" + movie_id + "_"+ genre_ids + ".jpg"
            
            urllib.urlretrieve(url, path)
            
            # to keep track of progress                   
            counter += 1
            print(counter, end=', ')

War
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, Crime
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, Music
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5

In [91]:
for genre_data in data.values():
    assert(len(genre_data) >= 100)

In [119]:
test_movie = data["Mystery"][0]

In [134]:
test_movie["overview"].encode("utf-8")

'Taking place after alien crafts land around the world, an expert linguist is recruited by the military to determine whether they come in peace or are a threat.'

In [137]:
test_movie["genre_ids"]

[53, 18, 878, 9648]

In [144]:
test_movie["title"]

u'Arrival'

In [ ]:
test_movie[]

## Put Movie Data in DataFrame

In [245]:
df = pd.DataFrame()
df["movie_id"] = pd.Series()
df["movie_title"] = pd.Series()
df["overview"] = pd.Series()
df["genre_id"] = pd.Series()

In [246]:
index = 0
for genre_data in data.values():
    for movie in genre_data:
        df.loc[index] = (int(movie["id"]), movie["title"].encode("utf-8"),
                          movie["overview"].encode("utf-8"),
                          "-".join(str(id) for id in movie["genre_ids"]))
        index += 1

In [247]:
df["movie_id"] = df["movie_id"].astype(int)
df.index = df['movie_id']
df.head()

,movie_id,movie_title,overview,genre_id
movie_id,,,,
329865,329865,Arrival,Taking place after alien crafts land around th...,53-18-878-9648
210577,210577,Gone Girl,With his wife's disappearance having become th...,9648-53-18
419430,419430,Get Out,A young black man visits his white girlfriend'...,27-9648-53
27205,27205,Inception,"Cobb, a skilled thief who commits corporate es...",28-53-878-9648-12
273248,273248,The Hateful Eight,Bounty hunters seek shelter from a raging bliz...,80-18-9648-37


## Split genre ID's and apply one-hot encoding



In [248]:
split_genres = [map(int, genre_strings.split("-")) for genre_strings in df["genre_id"]]

In [249]:
df["genre_id"] = split_genres

In [250]:
df.head()

,movie_id,movie_title,overview,genre_id
movie_id,,,,
329865,329865,Arrival,Taking place after alien crafts land around th...,"[53, 18, 878, 9648]"
210577,210577,Gone Girl,With his wife's disappearance having become th...,"[9648, 53, 18]"
419430,419430,Get Out,A young black man visits his white girlfriend'...,"[27, 9648, 53]"
27205,27205,Inception,"Cobb, a skilled thief who commits corporate es...","[28, 53, 878, 9648, 12]"
273248,273248,The Hateful Eight,Bounty hunters seek shelter from a raging bliz...,"[80, 18, 9648, 37]"


In [251]:
df["genre_id"].iloc[0]

[53, 18, 878, 9648]

In [252]:
# one-hot-encoding by genre
for genre_id, genre in genres.items():
    df[genre] = [int(genre_id in genre_list) for genre_list in df['genre_id']]

In [253]:
df.to_csv("full_raw_movie_overviews.csv", index=False)